# Fairness Evaluation

 El desarrollo y uso responsable de modelos de inteligencia artificial es un tema de gran interés para los gobiernos, corporaciones y organizaciones internacionales ya que ésta se ha convertido en una herramienta de gran apoyo para una gran variedad de sectores. De acuerdo a [*Dignum, V.*](https://arxiv.org/abs/2205.10785) y [*Jobin, A., et. al.*](https://www.nature.com/articles/s42256-019-0088-2) las directrices éticas de las aplicaciones de la inteligencia artificial publicadas en más de 600 artículos por diversas organizaciones, convergen a cinco principios:
 * Transparencia: 
 
    Comprenden los esfuerzos para aumentar la explicabilidad, la interpretabilidad u otros actos de comunicación y divulgación. Se presenta como una forma de minimizar el daño y mejorar la IA, aunque algunas fuentes subrayan su beneficio por razones legales o para fomentar la confianza. Algunas fuentes también relacionan la transparencia con el diálogo, la participación y los principios de la democracia


 * Justicia y equidad:

   La justicia se expresa principalmente en términos de equidad y de prevención, control o mitigación de prejuicios y discriminación no deseados. Algunas fuentes se centran en la justicia como respeto a la diversidad, la inclusión y la igualdad, otras reclaman la posibilidad de apelar o impugnar las decisiones, o el derecho a la reparación y el remedio. Las fuentes también destacan la importancia de un acceso justo a la IA, a los datos y a los beneficios de la IA.


 * No maleficiencia: 

    Se refiere a la necesidad de evitar los daños y los riesgos, o de prevenir y mitigar los daños y los riesgos no deseados. Se afirma que la IA nunca debe causar un daño previsible o involuntario.

    
 * Responsabilidad:
 
   La responsabilidad rara vez se define. No obstante, las recomendaciones específicas incluyen actuar con "integridad" y aclarar la atribución de responsabilidad y la responsabilidad legal si es posible por adelantado, en los contratos


 * Privacidad:

    La IA ética considera la privacidad como un valor que hay que defender y como un derecho que hay que proteger. Aunque a menudo no se define, la privacidad se presenta a menudo en relación con la protección y seguridad de los datos.


El principio de equidad puede evaluarse de forma cuantitativa a través de la segmentación de ciertos grupos de interés y las métricas precisión y exahustividad. Con el fin de evaluar la posible existencia de sesgo de equidad en la clasificación xenófoba se considerará la segmentación por país de los tweets pertenecientes al conjunto de prueba.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import classification_report as report

In [2]:
labeled_data = pd.read_parquet('merged_test_mongo04jul22.parquet')
labeled_data._id = labeled_data._id.astype(int)

In [3]:
labeled_data.is_mig.value_counts()

True     4909
False     806
Name: is_mig, dtype: int64

In [4]:
labeled_data.a_code.value_counts()

MEX    697
VEN    659
CHL    534
COL    424
PER    184
ARG    181
ECU    121
GTM     62
PAN     61
SLV     45
HND     41
URY     38
DOM     31
CRI     25
BOL     16
BRA     12
PRY     10
GUY      9
NIC      8
BLZ      1
JAM      1
USA      1
Name: a_code, dtype: int64

In [12]:
labeled_data[labeled_data.a_code.isna()]

,_id,text,date,label,predicted_xeno,xeno_probas,xeno_proba,a_code,is_mig
0,1234917304883765248,"@usuario Chorrillos, zona liberada de delincue...",2021-10-22,1,1,"[0.00042599125624483966, 0.9995740087437551]",0.999574,None,True
3,1301350292341039104,@usuario @usuario @usuario @usuario debe cuida...,2021-07-22,1,1,"[0.01774644115595672, 0.9822535588440433]",0.982254,None,True
4,1368124417679720448,"@usuario @usuario @usuario Voto sí, si se esta...",2021-10-11,1,1,"[0.001380022786645687, 0.9986199772133543]",0.998620,None,True
7,1186452316586680320,@usuario Lamentando que unos delincuentes mane...,2021-08-27,0,1,"[0.22405179645682916, 0.7759482035431708]",0.775948,None,False
8,1234832015414976512,@usuario Me imagino que de las manadas de inmi...,2021-09-04,0,0,"[0.9873672824689208, 0.012632717531079316]",0.987367,None,True
...,...,...,...,...,...,...,...,...,...
5702,1306657297310113792,@usuario @usuario @usuario Lo peor es que ese ...,2021-09-03,1,0,"[0.9311318930899787, 0.06886810691002132]",0.931132,None,False
5704,1211616197202063360,@usuario Van a aparecer los pro inmigración ir...,2021-07-28,0,1,"[0.005176695114690365, 0.9948233048853097]",0.994823,None,True
5705,1133745808883539968,"Lo mismo he pensado siempre ,vi ese vídeo y la...",2021-07-07,0,0,"[0.66300054464247, 0.33699945535752995]",0.663001,None,True
5713,1220365934247825408,@usuario @usuario @usuario url MATAD. MATADL...,2021-08-05,0,1,"[0.346095757607996, 0.6539042423920041]",0.653904,None,True


In [11]:
labeled_data[labeled_data.a_code.isna()].label.value_counts()

0    1936
1     618
Name: label, dtype: int64

In [ ]:
def plot_confusion_matrix(test_data, iso_code, country_name, norm=False):
    if iso_code:
        country_data = test_data[test_data.a_code==iso_code].reset_index(drop=True)
    else:
        country_data = test_data[test_data.a_code.isna()].reset_index(drop=True)
    
    confusion = confusion_matrix(y_true = country_data.label.to_list(), 
                                 y_pred = country_data.predicted_xeno.to_list(), 
                                 normalize='true' if norm else None)
    if norm:
        confusion = confusion*100
        confusion = np.around(confusion, 2)
    
    fig = px.imshow(confusion,
                labels=dict(y="Etiqueta", x="Predicción", color="Cantidad"),
                x=['No xenófobo', 'Xenófobo'],
                y=['No xenófobo', 'Xenófobo'], template='seaborn', text_auto=True,
                title=f'Matriz de confusión para el país {country_name}', width=650, height=650)
    fig.update_layout(font=dict(size=14))
    return fig

In [18]:
def print_report(test_data, iso_code, country_name):
    if iso_code:
        country_data = test_data[test_data.a_code==iso_code].reset_index(drop=True)
    else:
        country_data = test_data[test_data.a_code.isna()].reset_index(drop=True)
    target_names=['No xenófobo', 'Xenófobo']
    print(classification_report(y_true = country_data.label.to_list(), 
                                y_pred = country_data.predicted_xeno.to_list(),
                                target_names=target_names, digits=4))

In [ ]:
mex = plot_confusion_matrix(labeled_data, 'MEX', 'México', True)
ven = plot_confusion_matrix(labeled_data, 'VEN', 'Venezuela', True)
chl = plot_confusion_matrix(labeled_data, 'CHL', 'Chile', True)
col = plot_confusion_matrix(labeled_data, 'COL', 'Colombia', True)
per = plot_confusion_matrix(labeled_data, 'PER', 'Perú', True)
nda = plot_confusion_matrix(labeled_data, None, 'No identificado', True)

In [27]:
print_report(labeled_data, None, 'México')

              precision    recall  f1-score   support

 No xenófobo     0.9245    0.9168    0.9206      1936
    Xenófobo     0.7461    0.7654    0.7556       618

    accuracy                         0.8802      2554
   macro avg     0.8353    0.8411    0.8381      2554
weighted avg     0.8813    0.8802    0.8807      2554



In [ ]:
with open('fairness_eval_pct.html', 'a') as f:
    f.write(nda.to_html(full_html=False))
    f.write(mex.to_html(full_html=False))
    f.write(ven.to_html(full_html=False))
    f.write(chl.to_html(full_html=False))
    f.write(col.to_html(full_html=False))
    f.write(per.to_html(full_html=False))